In [22]:

import os
from PIL import Image
from datetime import datetime
import colorsys

# Define time ranges and corresponding hex color ranges
time_ranges = [
    ("Morning", (6, 12), ['#E7D48F', '#EDDB6F', '#e2cc0d']),
    ("Noon", (12, 15), ['#CEDCB8', '#B4F735', '#4AD83D']),
    ("Afternoon", (15, 18), ['#87B7B6', '#15C3C3', '#377D9B']),
    ("Evening", (18, 0), ['#EDB7CF', '#FD94CF', '#CC5CCC', '#CB21B5']),
    ("Night", (0, 6), ['#CFBCD0', '#93849F', '#5449C7', '#131F8F'])
]

def categorize_time_range(hour):
    for label, (start, end), _ in time_ranges:
        if (start <= end and start <= hour < end) or (start > end and (start <= hour or hour < end)):
            return label

def generate_hex_color(min_hex, max_hex, ratio):
    r1, g1, b1 = [int(min_hex[i:i + 2], 16) for i in (1, 3, 5)]
    r2, g2, b2 = [int(max_hex[i:i + 2], 16) for i in (1, 3, 5)]

    r = int(r1 + ratio * (r2 - r1))
    g = int(g1 + ratio * (g2 - g1))
    b = int(b1 + ratio * (b2 - b1))

    return "#{:02X}{:02X}{:02X}".format(r, g, b)

def analyze_images(directory):
    image_data = []

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(directory, filename)

            try:
                img = Image.open(filepath)
                exif_data = img._getexif()

                if exif_data and 36867 in exif_data:
                    capture_time = exif_data[36867]  # DateTimeOriginal tag
                    timestamp_type = "Original"
                else:
                    capture_time = datetime.fromtimestamp(os.path.getmtime(filepath)).strftime("%Y:%m:%d %H:%M:%S")
                    timestamp_type = "Modified"

                datetime_obj = datetime.strptime(capture_time, "%Y:%m:%d %H:%M:%S")
                hour = datetime_obj.hour
                time_category = categorize_time_range(hour)

                for label, (start, end), hex_range in time_ranges:
                    if label == time_category:
                        ratio = (hour - start) / (end - start)
                        hex_color = generate_hex_color(hex_range[0], hex_range[-1], ratio)
                        image_data.append({
                            "image": filename,
                            "time": datetime_obj.strftime("%H:%M"),
                            "timestamp_type": timestamp_type,
                            "category": time_category,
                            "hex_color_code": hex_color
                        })
                        break
            except (AttributeError, KeyError, ValueError) as e:
                print(f"Error processing image {filename}: {e}")
                pass

    return image_data

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/picss"
    output_data = analyze_images(input_directory)

    for entry in output_data:
        print(entry)

{'image': 'summer-image.jpg', 'time': '20:16', 'timestamp_type': 'Modified', 'category': 'Evening', 'hex_color_code': '#F0C7D1'}
{'image': '5744.jpg', 'time': '20:16', 'timestamp_type': 'Modified', 'category': 'Evening', 'hex_color_code': '#F0C7D1'}
{'image': 'images.jpeg', 'time': '20:16', 'timestamp_type': 'Modified', 'category': 'Evening', 'hex_color_code': '#F0C7D1'}
{'image': '20211013_154646.jpg', 'time': '15:46', 'timestamp_type': 'Original', 'category': 'Afternoon', 'hex_color_code': '#87B7B6'}
{'image': 'IMG-20230710-WA0006.jpg', 'time': '09:28', 'timestamp_type': 'Modified', 'category': 'Morning', 'hex_color_code': '#E4D04E'}
{'image': 'IMG-20230710-WA0004.jpg', 'time': '09:28', 'timestamp_type': 'Modified', 'category': 'Morning', 'hex_color_code': '#E4D04E'}
{'image': 'IMG-20230710-WA0007.jpg', 'time': '09:28', 'timestamp_type': 'Modified', 'category': 'Morning', 'hex_color_code': '#E4D04E'}
{'image': 'IMG-20230709-WA0000.jpg', 'time': '09:28', 'timestamp_type': 'Modified', 